Neural Machine Translation by Jointly Learning to Align and Translate https://arxiv.org/abs/1409.0473

Attention? Attention! https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html



When you first look at an image, you first focus on one spot before moving to another spot. This wandering focus is your attention: it spotlights the things that matter, and ignores the things that don't.

Just as you can divide an image into sections (spots), and selectively process on just a few of them, you can do the same thing with words, where each word token is its own section.

Besides scanning from spot to spot on an image, your mind also 'expects' (predicts) what it'll see next based on the things it has already seen. For example, when you see a furry texture with a snout, your mind expects to see an animal.

With words, if you read 'she is eating a ...', you'd expect that whatever follows is a food of some sort.

So attention is not just the focus on where to look, but also how the item you're looking at change your expectation.

a new context vector is generated by the decoder before it generates an output. the context vector is calculated from the annotations and the annotation weights.

when the decoder is trying to generate an output $y_t$, it needs to first figure out which input word to look at. This is done by 

[The paper](https://arxiv.org/abs/1409.0473)

divide the input image into grids

use a weight to decide which block to look at

you can do the same to a rnn for words:

retain all the hidden states, and stack them into a matrix

apply a weight to select which hidden state to look at when the decoder is generating a translation

### The encoder

the input sentence is run twice: once forwrad, once backward into the rnn. the hidden states of each run are collected and lined up according to the order of the words. 

now for each word in the sentence, you have 2 hidden states: one that remembers the words before it, and the other remembers the words that come after it.

The run from the first word $x_1$ to the last word $x_T$ to calculate the forward hidden states:

$$(\overrightarrow{h_1}, \dots, \overrightarrow{h_{T_x}})$$

The backward run from the last word $x_T$ to the first word $x_1$ to calculate the backward hidden states:

$$(\overrightarrow{h_{T_x}}, \dots, \overrightarrow{h_1})$$

This backward hidden states vector is reversed so that the order of inputs $x_t$ are aligned with the forward hidden states vector:

$$(\overleftarrow{h_1}, \dots, \overleftarrow{h_{T_x}})$$

the forward and backward hidden states for each input word are stacked (concatenated) together to form its __annotation__ $h_j$:

$$h_j = \left[
\overrightarrow{h}_j^\top ; \overleftarrow{h}_j^\top \right]^\top$$

Since RNNs only retain recent knowledge, the combined hidden states at each step will have information for both the preceding words, and the following words.

### The decoder

The decoder decides parts of the source sentence to pay attention to.

#### The attention

To find which words (annotations) to pay attention to, we need some sort of mask to highlight certain words, and diminish others before the decoder can decide what to output.

This mask is the context vector $c_i$

$$c_i = \sum_{j=1}^{T_x} \alpha_{ij} h_j$$

The context vector __for the current step__ $c_i$ is the sum of the current step's alignment weight $a_{ij}$ multiplied by all annotations $h_j$.

_Alignment_ means matching the input word (annotation) to the output word. The alignment is based on the decoder's previous hidden state $s_{i-1}$ and the annotation $h_j$.

__$a_{ij}$ selects which words to pay attention to by highlighting some words, and diminishing others__.

$a_{ij}$ is the probability that output $y_i$ is aligned to the source word $x_j$.

For each output $y_i$, an annotation weight $a_{ij}$ is generated for every word $x_{j}$ of the source sentence.

Once the decoder has the context vector, it can go on to calculate the new hidden state $s_i$ and an output $y_i$:

$$s_i = f(s_{i-1}, y_{i-1}, c_i)$$

$$y_i = g(s_i, y_{i-1}, c_i)$$

#### TODO

I still don't fully understand how the annotation weight $a_{ij}$ is calculated. The paper lists these 2 equations which I still can't parse in my head (in plain language), so I'll leave them here and update my notes later when I finally figure them out.

Copied verbatim from the paper:

$$\alpha_{ij} = \frac{\exp\left(e_{ij}\right)}{\sum_{k=1}^{T_x} \exp\left(e_{ik}\right)}$$

where

$$e_{ij} = a(s_{i-1}, h_j)$$

is an _alignment model_ which scores how well the inputs around position $j$ and the output at position $i$ match. The score is based on the decoder's last hidden state $s_{i-1}$

#### self-attention == association